# TL-Test: Cache 验证框架
**TL-Test: Cache Verification Framework**

Cache 系统与上游模块的协同验证复杂，且不利于快速迭代与快速定位问题。

Co-verification of the Cache system with upstream modules is complex and prevents rapid iteration.

为了解决这个问题，我们开发了 TL-Test - 一个单元级的缓存系统验证框架，TL-Test 支持Tilelink总线协议以及cache一致性检查，同时支持随机测试用例生成。
  
To address this issue, we developed TL-Test: a unit-level cache-system verification framework that supports the TileLink protocol, cache-coherence checking, and randomized test-case generation.

![TL-TEST](../images/02-functional/05-tl_test/tl-test-overall-en.png)

这里有一个例子演示使用 TL-Test 发现缓存一致性违例：我们插入了一个错误移位 Grant data 的 BUG。

Here is another example to detect cache coherence violation by TL-Test. We inject a bug that wrongly shift the grant data.



In [ ]:
%%bash
cd ../ && source env.sh

cat $(get_asset tltest-precompile/tlt_err.patch)

TL-Test 会生成随机测试，然后发现我们的 cache 设计在某个地址上的传输存在问题。TL-Test 会将所有的总线传输事务转为 log，我们可以通过 grep 将错误地址的总线事务筛选出来进行分析。

TL-Test generates randomized tests and pinpoints a transfer problem at a specific address in our cache design. It logs all bus transactions; we then use grep to extract those related to the offending address for analysis.

In [ ]:
%%bash
cd ../ && source env.sh
# cd $TLT_HOME && make coupledL2-test-l2l3-v3 run THREADS_BUILD=16 CXX_COMPILER=clang++-17
# cd $TLT_HOME/run && ./tltest_v3lt 2>&1 | tee tltest_v3lt.log

get_asset tltest-precompile/tltest_err

mkdir -p ${WORK_DIR}/02-functional/05-tltest

cd ${WORK_DIR}/02-functional/05-tltest

cp -r $(get_asset tltest-precompile) ./ && cd ./tltest-precompile

./tltest_err 2>&1 | tee tltest_v3lt.log

产生错误的地址为：0x80

Error Addr： 0x80

In [ ]:
%%bash
cd ../ && source env.sh > /dev/null
# grep "addr: 0x80" $TLT_HOME/run/tltest_v3lt.log

cd ${WORK_DIR}/02-functional/05-tltest/tltest-precompile && grep "addr: 0x80," tltest_v3lt.log

Columns: [time] [INFO-level] #nodeIdx core [channel] [opcode] source, address, alias, data

1. L1D acquires Eaddr
```
[236] [tl-test-new-INFO] #0 L2[0].C[0] [fire A] [AcquirePerm NtoT] source: 0x3, addr: 0x80, alias: 0
```
2. L1D release Eaddr, and data successfully transferred from L1D to L2
```
[806] [tl-test-new-INFO] #0 L2[0].C[0] [fire C] [ReleaseData TtoN] source: 0x3, addr: 0x80, alias: 0, data: [ c7 a5 ... ]
[808] [tl-test-new-INFO] #0 L2[0].C[0] [fire C] [ReleaseData TtoN] source: 0x3, addr: 0x80, alias: 0, data: [ fe 14 ... ]
```
3. Next time L1D acquires Eaddr, L2 returns wrong data
```
[2036] [tl-test-new-INFO] #0 L2[0].C[0] [fire D] [GrantData toT] source: 0xf, addr: 0x80, alias: 0x1, data: [ 00 c7 ... ]
[2038] [tl-test-new-INFO] #0 L2[0].C[0] [fire D] [GrantData toT] source: 0xf, addr: 0x80, alias: 0x1, data: [ 00 fe ... ]
```